[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pucrs-automated-planning/tabular-rl/blob/main/src/Tabular-Methods.ipynb)

# Reinforcement Learning

## Tutorial 02: Tabular Methods

#### Prof. Felipe Meneguzzi

### Introduction

In this practical, you will implement two tabular methods of reinforcement learning: [Q-Learning](https://en.wikipedia.org/wiki/Q-learning) and [Sarsa](https://en.wikipedia.org/wiki/State–action–reward–state–action) according to Sutton and Barto's book. Our implementation will be based on a simple [RLAgent](rl/agent.py) class provided by the instructor, and the simplest of the [Gym](https://github.com/openai/gym) environments.

<div class="alert alert-block alert-warning">
You will need to configure your environment following the instructions in the `README.md` file included in this repository.
</div>

In [ ]:
try:
  import google.colab
  print("We are in Google colab, we need to clone the repo")
  !git clone https://github.com/pucrs-automated-planning/tabular-rl.git
  %cd tabular-rl
  %pip install -r requirements.txt
  %cd src
except:
  print("Not in colab")

## The agent interface

In this notebook we will use a basic agent interface called [`RLAgent`](src/rl/agent.py). Open this file and become familiar with its key attributes and methods. 

The key attributes you will need for this notebook are the following:
- `self.env` — a reference to the [environment from gym](https://github.com/openai/gym/blob/58ed658d9b15fd410c50d1fdb25a7cad9acb7fa4/gym/core.py#L8);
- `self.episodes` — the number of episodes to train the agent when invoking [`RLAgent.learn()`](rl/agent.py#L125);
- `self.alpha` — the *learning rate* to be used when computing the RL update;
- `self.gamma` — the *discount factor* for the return computed during training; and
- `self.q_table` — a dictionary of lists of floating point numbers indexed by the state and an action (e.g., `self.q_table[state][action]` should return the q-value of the `state`-`action` pair)
- `self.actions` — an integer representing the size of the action space
- `self.last_state` — the last state visited by the agent during learning
- `self.last_action` — the last action chosen by the agent during learning
- `self._random` — a reference to a Python [Random](https://docs.python.org/3/library/random.html#random.Random) object.

Pay attention to the `self.random` object, and use it instead of the unbound methods from Python's [random library](https://docs.python.org/3/library/random.html), otherwise the tests will not work. 

The key methods you should be aware of are the following:
- [`policy`](rl/agent.py#L90) — returns the greedy policy following the currently learned q-values;
- [`epsilon_greedy_policy`](rl/agent.py#L101) — returns an $\epsilon$-greedy policy;
- [`agent_start`](rl/agent.py#L55) — the method to be called once *before each episode*;
- [`agent_step`](rl/agent.py#L67) — the method to be called once at *each time step* within an episode;
- [`agent_end`](rl/agent.py#L80) — the method to be called once at the *end of each episode* when the agent reaches a *terminal state*; and
- [`learn`](rl/agent.py#L125) — runs the underlying learning algorithm for `self.episodes` episodes

## Greedy and $\epsilon$-Greedy Policies

In the cell below, you need to implement both the greedy and the $\epsilon$-greedy policy using the q-values stored in `self.q_table` and the $\epsilon$ from `self.eps()`. 

Recall the definition of $\epsilon$-greedy policy:

For all $a \in \mathcal{A}$
$$\pi(a \mid S_t) = \begin{cases} 1 - \epsilon +\frac{\epsilon}{|\mathcal{A}|} & \text{if}~a = A^* \\ \frac{\epsilon}{|\mathcal{A}|} & \text{if}~a \neq A^*\end{cases}$$
where $A^* = \arg\max_{a}Q(S_t,a)$, with ties broken randomly (this is the greedy policy).

In [ ]:
import gym
import numpy as np
from rl.agent import RLAgent, Action, State
from random import Random
from gym import Env
from typing import Any, Collection, List, NoReturn, Sequence

class TabularQLearner(RLAgent):
    """
    A simple Tabular Q-Learning agent.
    """
    def __init__(self,
                 env: Env,
                 episodes: int = 500,
                 decaying_eps: bool = True,
                 eps: float = 1.0,
                 alpha: float = 0.5,
                 decay: float = 0.000002,
                 gamma: float = 0.9,
                 rand: Random = Random(),
                 **kwargs):
        super().__init__(env, episodes=episodes, decaying_eps=decaying_eps, eps=eps, alpha=alpha, decay=decay, gamma=gamma, rand=rand)
        self.actions = env.action_space.n
        self.q_table = {}

        # hyperparameters
        self.episodes = episodes
        self.gamma = gamma
        self.decay = decay
        self.c_eps = eps
        self.base_eps = eps
        if decaying_eps:
            def epsilon():
                self.c_eps = max((self.episodes - self.step)/self.episodes, 0.01)
                return self.c_eps
            self.eps = epsilon
        else:
            self.eps = lambda: eps
        self.decaying_eps = decaying_eps
        self.alpha = alpha
        self.last_state = None
        self.last_action = None
    
    def add_new_state(self, state: State):
        # self.q_table[state] = [1. for _ in range(self.actions)]
        self.q_table[state] = [0.]*self.actions

    def get_all_q_values(self, state: State) -> List[float]:
        if state in self.q_table:
            return self.q_table[state]
        else:
            return [0.]*self.actions

    def best_actions(self, state: State) -> Sequence[int]:
        """Returns a list with the best actions for a particular state

        Args:
            state (State): The state for which we need the best actions

        Returns:
            Sequence[int]: a list with the best actions
        """
        if state not in self.q_table:
            self.add_new_state(state)
        q_next = np.array(self.q_table[state])
        best_actions = np.argwhere(q_next == np.max(q_next)).flatten()
        return best_actions

    def best_action(self, state: State) -> Action:
        """Returns the best action for a state, breaking ties randomly

        Args:
            state (State): The state in which to extract the best action

        Returns:
            int: The index of the best action
        """
        if state not in self.q_table:
            self.add_new_state(state)
            # self.q_table[state] = [0 for _ in range(self.actions)]
        # return np.argmax(self.q_table[state])
        best_actions = self.best_actions(state)

        return self._random.choice(best_actions)
    
    def policy(self, state: State) -> Action:
        """Returns the greedy deterministic policy for the specified state

        Args:
            state (State): the state for which we want the action

        Raises:
            InvalidAction: Not sure about this one

        Returns:
            Any: The greedy action learned for state
        """
        ## YOUR CODE HERE (1 line)
        pass
        ## END CODE

    def epsilon_greedy_policy(self, state: State) -> Action:
        """Returns the epsilon-greedy policy

        Args:
            state (State): The state for which to return the epsilon greedy policy

        Returns:
            Any: The action to be taken
        """
        ## YOUR CODE HERE (6 lines)
        

        
        
        
        pass
        ## END CODE

In [ ]:
env = gym.make("CliffWalking-v0")
agent = TabularQLearner(env = env, decaying_eps=False, eps=0.5, rand = Random(42))
agent.q_table["a"] = [1, 0.5, 4, 0]
agent.q_table["b"] = [0, 1, 0, 0.5]

action_a = agent.epsilon_greedy_policy("a")
action_b = agent.epsilon_greedy_policy("b")
print(f"Action for a={action_a}")
print(f"Action for b={action_b}")

assert(action_a == 2)
assert(action_b == 1)

action_a = agent.epsilon_greedy_policy("a")
action_b = agent.epsilon_greedy_policy("b")

print(f"Action for a={action_a}")
print(f"Action for b={action_b}")

assert(action_a == 0)  # With our random seed, this one should be exploratory
assert(action_b == 1)

## Q-Learning

**Description** 

Q-Learning is an off policy algorithm that learns the target policy by temporal-difference from a greedy target policy using the following update rule. In our implementation, we will use the $\epsilon$-greedy policy as the source policy. 

$$ Q(S,A) \gets Q(S,A) + \alpha \left(R + \gamma\max_{a'}Q(S',a') - Q(S,A) \right) $$ 

In the cell below, you should implement the following methods (as noted in the source code):
- `policy` and `epsilon_greedy_policy` - copy the code you just developed above
- `agent_start` - initialize the last state with the initial state given by the environment and choose one initial action
- `agent_step` - this should follow the algorithm in Section 6.5 from Sutton and Barto
- `agent_end` - one last update at the end of the episode, but using just the old value of Q as the TD-error

In [ ]:
class TabularQLearner(RLAgent):
    """
    A simple Tabular Q-Learning agent.
    """
    def __init__(self,
                 env: Env,
                 episodes: int = 500,
                 decaying_eps: bool = True,
                 eps: float = 1.0,
                 alpha: float = 0.5,
                 decay: float = 0.000002,
                 gamma: float = 0.9,
                 rand: Random = Random(),
                 **kwargs):
        super().__init__(env, episodes=episodes, decaying_eps=decaying_eps, eps=eps, alpha=alpha, decay=decay, gamma=gamma, rand=rand)
        self.actions = env.action_space.n
        self.q_table = {}

        # hyperparameters
        self.episodes = episodes
        self.gamma = gamma
        self.decay = decay
        self.c_eps = eps
        self.base_eps = eps
        if decaying_eps:
            def epsilon():
                self.c_eps = max((self.episodes - self.step)/self.episodes, 0.01)
                return self.c_eps
            self.eps = epsilon
        else:
            self.eps = lambda: eps
        self.decaying_eps = decaying_eps
        self.alpha = alpha
        self.last_state = None
        self.last_action = None

    def add_new_state(self, state: State):
        # self.q_table[state] = [1. for _ in range(self.actions)]
        self.q_table[state] = [0.]*self.actions

    def get_all_q_values(self, state: State) -> List[float]:
        if state in self.q_table:
            return self.q_table[state]
        else:
            return [0.]*self.actions

    def best_actions(self, state: State) -> Sequence[int]:
        """Returns a list with the best actions for a particular state

        Args:
            state (State): The state for which we need the best actions

        Returns:
            Sequence[int]: a list with the best actions
        """
        if state not in self.q_table:
            self.add_new_state(state)
        q_next = np.array(self.q_table[state])
        best_actions = np.argwhere(q_next == np.max(q_next)).flatten()
        return best_actions

    def best_action(self, state: State) -> Action:
        """Returns the best action for a state, breaking ties randomly

        Args:
            state (State): The state in which to extract the best action

        Returns:
            int: The index of the best action
        """
        if state not in self.q_table:
            self.add_new_state(state)
            # self.q_table[state] = [0 for _ in range(self.actions)]
        # return np.argmax(self.q_table[state])
        best_actions = self.best_actions(state)

        return self._random.choice(best_actions)
    
    def get_max_q(self, state: State) -> float:
        if state not in self.q_table:
            self.add_new_state(state)
        return np.max(self.q_table[state])
    
    def set_q_value(self, state: State, action: Any, q_value: float):
        if state not in self.q_table:
            self.add_new_state(state)
        self.q_table[state][action] = q_value

    def get_q_value(self, state: State, action: Any) -> float:
        if state not in self.q_table:
            self.add_new_state(state)
        return self.q_table[state][action]

    
    def policy(self, state: State) -> Action:
        """Returns the greedy deterministic policy for the specified state

        Args:
            state (State): the state for which we want the action

        Raises:
            InvalidAction: Not sure about this one

        Returns:
            Any: The greedy action learned for state
        """
        ## YOUR CODE HERE (1 line)
        pass
        ## END CODE

    def epsilon_greedy_policy(self, state: State) -> Action:
        """Returns the epsilon-greedy policy

        Args:
            state (State): The state for which to return the epsilon greedy policy

        Returns:
            Any: The action to be taken
        """
        ## YOUR CODE HERE (6 lines)
        
        
        
        
        
        pass
        ## END CODE
    
    def agent_start(self, state: State) -> Action:
        """The first method called when the experiment starts,
        called after the environment starts.
        Args:
            state (Numpy array): the state from the
                environment's env_start function.
        Returns:
            (int) the first action the agent takes.
        """
        ## YOUR CODE HERE (3 lines)
        
        
        pass
        ## END CODE

    def agent_step(self, reward: float, state: State) -> Action:
        """A step taken by the agent.

        Args:
            reward (float): the reward received for taking the last action taken
            state (Any): the state from the
                environment's step based on where the agent ended up after the
                last step
        Returns:
            (int) The action the agent takes given this state.
        """
        ## YOUR CODE HERE (9 lines)
        
        
        
        
        
        
        
        
        pass
        ## END CODE

    def agent_end(self, reward: float) -> NoReturn:
        """Called when the agent terminates.

        Args:
            reward (float): the reward the agent received for entering the
                terminal state.
        """
        ## YOUR CODE HERE (4 lines)
        
        
        
        pass
        ## END CODE

In [ ]:
# Test for Q-Learning
## Mini test
nv = gym.make("CliffWalking-v0")
agent = TabularQLearner(env = env, decaying_eps=False, eps=0.5, rand = Random(42))
agent.q_table["a"] = [1, 0.5, 4, 0]
agent.q_table["b"] = [0, 1, 0, 0.5]

action_a = agent.agent_start("a")
assert(action_a == 2)

action_a = agent.agent_step(-1, "a")
# print(action_a)
# print(agent.q_table["a"])
# print(agent.q_table["b"])
assert(action_a == 2)
assert(agent.q_table["a"] == [1, 0.5, 3.3, 0])
assert(agent.q_table["b"] == [0, 1, 0, 0.5])

action_b = agent.agent_step(-2, "b")
# print(action_b)
# print(agent.q_table["a"])
# print(agent.q_table["b"])
assert(action_b == 0)
assert(np.where(np.isclose(agent.q_table["a"], [1, 0.5, 1.1, 0])))
assert(agent.q_table["b"] == [0, 1, 0, 0.5])

action_b = agent.agent_step(1, "b")
# print(action_b)
# print(agent.q_table["a"])
# print(agent.q_table["b"])
assert(action_b == 1)
assert(np.where(np.isclose(agent.q_table["a"], [1, 0.5, 1.1, 0])))
assert(agent.q_table["b"] == [0.95, 1, 0, 0.5])


## The Cliff Walking Environment
We are now ready to learn our first real policy for a non-trivial environment. At the end of training, we print the resulting policy. 

The environment we use is the Cliff Walking environment from Example 6.6 in Sutton and Barto's book. Refer to page 132 for further details, but we post a summary of the environment below for your convenience.
!["Cliff Sarsa"](cliff-sarsa.svg)

Why does the policy look the way it does (i.e., a bit random at times)?

In [ ]:
##
from rl.env_utils import display_cliffwalking_policy
env = gym.make("CliffWalking-v0")
q_agent = TabularQLearner(env, episodes=500, gamma=1, eps=0.1, alpha=0.5, decaying_eps=False, rand=Random(42))
rewards_q = q_agent.learn()

print(f"Accumulated Reward: {np.sum(rewards_q)}")
# assert(np.sum(rewards_q) == -28481)
display_cliffwalking_policy(q_agent, env)

## Sarsa

Sarsa is an on policy algorithm where the source and target policy are the same. In our implementation, we will use the $\epsilon$-greedy policy we implemented above.

$$ Q(S,A) \gets Q(S,A) + \alpha \left(R + \gamma Q(S',A') - Q(S,A) \right) $$ 

Note that we can implement Sarsa by simply subclassing the `TabularQLearner` class we just implemented. This time, the only modification we need is in the `agent_step` method. 

In [ ]:
class TabularSarsaLearner(TabularQLearner):
    def agent_step(self, reward: float, state: State) -> Action:
        """A step taken by the agent.

        Args:
            reward (float): the reward received for taking the last action taken
            state (Any): the state from the
                environment's step based on where the agent ended up after the
                last step
        Returns:
            (int) The action the agent takes given this state.
        """
        ## YOUR CODE HERE (9 lines)
        
        
        
        
        
        
        
        
        pass
        ## END CODE

In [ ]:
# Test for Sarsa

v = gym.make("CliffWalking-v0")
sarsa_agent = TabularSarsaLearner(env = env, decaying_eps=False, eps=0.5, rand = Random(42))
sarsa_agent.q_table["a"] = [1, 0.5, 4, 0]
sarsa_agent.q_table["b"] = [0, 1, 0, 0.5]

action_a = sarsa_agent.agent_start("a")
assert(action_a == 2)

action_a = sarsa_agent.agent_step(-1, "a")
# print(action_a)
# print(sarsa_agent.q_table["a"])
# print(sarsa_agent.q_table["b"])
assert(action_a == 2)
assert(sarsa_agent.q_table["a"] == [1, 0.5, 3.3, 0])
assert(sarsa_agent.q_table["b"] == [0, 1, 0, 0.5])

action_b = sarsa_agent.agent_step(-2, "b")
# print(action_b)
# print(sarsa_agent.q_table["a"])
# print(sarsa_agent.q_table["b"])
assert(action_b == 0)
assert(np.where(np.isclose(sarsa_agent.q_table["a"], [1, 0.5, 1.1, 0])))
assert(sarsa_agent.q_table["b"] == [0, 1, 0, 0.5])

action_b = sarsa_agent.agent_step(1, "b")
# print(action_b)
# print(sarsa_agent.q_table["a"])
# print(sarsa_agent.q_table["b"])
assert(action_b == 1)
assert(np.where(np.isclose(sarsa_agent.q_table["a"], [1, 0.5, 1.1, 0])))
assert(sarsa_agent.q_table["b"] == [0.95, 1, 0, 0.5])

## Expected Sarsa

Expected Sarsa is a variation of the Sarsa algorithm where, instead of computing the TD target using the actual samples from a stochastic source policy, we compute the expectation from that policy, much like the update rule in Dynamic Programming, using the following update

$$ Q(S,A) \gets Q(S,A) + \alpha \left(R + \gamma \sum_{a'}\pi(a'\mid S')Q(S',a') - Q(S,A) \right) $$ 

Like Sarsa, we can implement Expected Sarsa by simply subclassing the `TabularQLearner` class we just implemented. This time, the only modification we need is in the `agent_step` method. 

In [ ]:
class TabularExpectedSarsaLearner(TabularQLearner):
    def agent_step(self, reward: float, state: State) -> Action:
        """A step taken by the agent.

        Args:
            reward (float): the reward received for taking the last action taken
            state (Any): the state from the
                environment's step based on where the agent ended up after the
                last step
        Returns:
            (int) The action the agent takes given this state.
        """
        ## YOUR CODE HERE (17 lines)
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        pass
        ## END CODE

In [ ]:
# Test for Expected Sarsa

v = gym.make("CliffWalking-v0")
expected_sarsa_agent = TabularExpectedSarsaLearner(env = env, decaying_eps=False, eps=0.5, rand = Random(42))
expected_sarsa_agent.q_table["a"] = [1, 0.5, 4, 0]
expected_sarsa_agent.q_table["b"] = [0, 1, 0, 0.5]

action_a = expected_sarsa_agent.agent_start("a")
assert(action_a == 2)

action_a = expected_sarsa_agent.agent_step(-1, "a")
print(action_a)
print(expected_sarsa_agent.q_table["a"])
print(expected_sarsa_agent.q_table["b"])
assert(action_a == 2)
assert(np.where(np.isclose(expected_sarsa_agent.q_table["a"], [1, 0.5, 2.7, 0])))
assert(expected_sarsa_agent.q_table["b"] == [0, 1, 0, 0.5])

action_b = sarsa_agent.agent_step(-2, "b")
print(action_b)
print(expected_sarsa_agent.q_table["a"])
print(expected_sarsa_agent.q_table["b"])
assert(action_b == 1)
assert(np.where(np.isclose(expected_sarsa_agent.q_table["a"], [1, 0.5, 2.7, 0])))
assert(expected_sarsa_agent.q_table["b"] == [0, 1, 0, 0.5])

action_b = expected_sarsa_agent.agent_step(1, "b")
print(action_b)
print(agent.q_table["a"])
print(agent.q_table["b"])
assert(action_b == 0)
assert(np.where(np.isclose(expected_sarsa_agent.q_table["a"], [1, 0.5, 1.1, 0])))
assert(np.where(np.isclose(expected_sarsa_agent.q_table["b"], [0.95, 1, 0, 0.5])))

# Comparing Learning Methods

Our final experiment compares the accumulated rewards for each of our TD learning algorithms. This experiment replicates exactly example 6.6 (plus Expected Sarsa) in the book by plotting the cumulative rewards per time step of training each of our algorithms achieves in the Cliff Walking environment. 

If your implementation is correct, you should see a graph like the one below:
!["Cumulative Rewards"](figure_example_6_6.svg).

In [ ]:
import matplotlib.pyplot as plt
env = gym.make("CliffWalking-v0")
test_episodes = 500
runs = 50
rewards_q = np.zeros(test_episodes)
rewards_sarsa = np.zeros(test_episodes)
rewards_expected_sarsa = np.zeros(test_episodes)
for i in range(runs):
    q_agent = TabularQLearner(env, episodes=test_episodes, gamma=1, eps=0.1, alpha=0.5, decaying_eps=False)
    sarsa_agent = TabularSarsaLearner(env, episodes=test_episodes, gamma=1, eps=0.1, alpha=0.5, decaying_eps=False)
    expected_sarsa_agent = TabularExpectedSarsaLearner(env, episodes=test_episodes, gamma=1, eps=0.1, alpha=0.5, decaying_eps=False)
    rewards_q += q_agent.learn()
    rewards_sarsa += sarsa_agent.learn()
    rewards_expected_sarsa += expected_sarsa_agent.learn()

rewards_q /= runs
rewards_sarsa /= runs
rewards_expected_sarsa /= runs

# draw reward curves
plt.plot(rewards_sarsa, label='Sarsa')
plt.plot(rewards_q, label='Q-Learning')
plt.plot(rewards_expected_sarsa, label='Expected Sarsa')
plt.xlabel('Episodes')
plt.ylabel('Sum of rewards during episode')
plt.ylim([-100, 0])
plt.legend()

plt.show()